# Atividade Prática 6

## Análise dinâmica de sinais

### Entrega: até 12/11/2021 às 23:59 no e-disciplinas

#### Nome: Lourenço Henrique Moinheiro Martins Sborz Bogo N° USP: 11208005  (X) Grad ( ) Pós

###  Sinais com conteúdo espectral variante no tempo

Todos os sinais no mundo real possuem conteúdo espectral que varia com o tempo: eventos começam e terminam, componentes senoidais que estavam presentes num certo momento já não estão presentes em outro, ou estão presentes com intensidade menor.

Em oposição a essa visão *dinâmica* do conteúdo dos sinais, a análise de Fourier em $\mathbb{C}^N$ parte de coleções de formas básicas de onda $\{E_k\}$ com frequências fixas $k=0,\ldots,N-1$, que para cada sinal $x$ analisado são associadas a coeficientes $X_k$ que são *estáticos*, visto que multiplicam "globalmente" o sinal temporal $E_k$.

Uma maneira de tornar a análise de Fourier mais "dinâmica" consiste em analisar segmentos curtos de um sinal "longo" $x\in\mathbb{C}^N$, tomando-se $M\ll N$ amostras de cada vez. Essa abordagem segmentada já apareceu no capítulo 3 e na atividade prática 4, tanto em sinais unidimensionais quanto bidimensionais (onde os segmentos eram chamados de "blocos" no esquema de compressão JPEG).

Usando dois parâmetros, $M>0$ (duração em amostras) e $m>0$ (amostra inicial), podemos determinar um segmento do sinal $x$ como

$$\tilde{x} = \left(x_{m},x_{m+1},\ldots,x_{m+M-1}\right)\in\mathbb{C}^M,$$

e assim considerar sua DFT $\tilde{X}\in\mathbb{C}^M$ para efeito de caracterização do conteúdo espectral de $x$ no recorte $[m,m+M)$.

Lembre-se que o número de amostras $M$ determina tanto a duração do sinal (de $\Delta_t=\frac{M}{R}$ segundos se $R$ é a taxa de amostragem) quanto a resolução em frequência (pois os bins da DFT são separados por $\Delta_f=\frac{R}{M}$ Hz).

Assim, numa análise segmentada, deseja-se usar segmentos de duração $\Delta_t$ razoavelmente curta para evitar que o caráter "estático" das formas básicas de onda de Fourier seja um empecilho, e suficientemente longos a fim de garantir uma resolução em frequência $\Delta_f$ adequada.

Esses dois critérios antagônicos estão na origem de uma questão crucial em análise de sinais que é frequentemente chamada de *Princípio da Incerteza*, por uma alusão metafórica ao princípio de Heisenberg em Física Quântica: não é possível localizar de forma perfeita uma componente do sinal simultaneamente nos domínios temporal e espectral:

- quanto melhor a localização temporal (menor $M$), maior a imprecisão espectral ($R/M$), e 

- quanto melhor a precisão espectral (menor $R/M$), pior a localização temporal (maisr $M$).

Em outros termos, a precisão temporal $\Delta_t=\frac{M}{R}$ e a resolução espectral $\Delta_f=\frac{R}{M}$ satisfazem a condição invariante $\Delta_t\Delta_f=1$.

### Exemplo: varredura senoidal

Na célula abaixo está construído um sinal, amostrado a $R=44100$ Hz, com uma varredura senoidal definida pela expressão

$$x(t) = \sin(2\pi f(t)t),\quad t\in[0,D)$$

onde $D=10$ segundos e $f(t)$ é uma função linear que vai de $0$ até $1000$ no intervalo $t\in[0,D)$. O espectro de magnitude desse sinal aparece em seguida, evidenciando o problema da falta de localização da DFT: várias frequências compõem o sinal, mas não se sabe em que trecho do sinal elas aparecem ou desaparecem.

In [ ]:
# importa dependências
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
from IPython.display import Audio
from ipywidgets import IntSlider, interactive, VBox, HBox

In [ ]:
R = 44100 # taxa de amostragem em Hz
D = 10 # duração em segundos
t = np.arange(0,D,1/R) # linha do tempo de 0 a D segundos
f = t*100 # função f(t)
x = np.sin(2*math.pi*f*t) # sinal x(t)
X = fft(x) # espectro de x
k0 = int(len(X)*2500/R) # índice correspondente a 2500 Hz
plt.plot(np.arange(k0)*R/len(X),abs(X[0:k0]))
plt.xlabel("frequência (Hz)")
plt.title(r"espectro de magnitude de x entre 0 e 2500 Hz")
Audio(x.T, rate=R) # elemento interativo para ouvir a varredura senoidal

**Exercício 1 (cont.):**

Adapte o código fornecido do exemplo interativo abaixo para representar o **espectro de magnitude**, sempre no intervalo de frequências de $0$ Hz a $2500$ Hz, do segmento do sinal $x$ definido pelos parâmetros interativos $m\in\{0,1000,2000,\ldots\}$ e $M\in\{100,200,300,\ldots,10000\}$. Em seguida, responda de forma sucinta as perguntas contidas na caixa de texto após o código.

**Dica:** *Cuidado com o último segmento, para garantir que $m+M-1$ ainda é um índice válido. Um exemplo de espectro no formato esperado é <img src="http://www.ime.usp.br/~mqz/dsp/espectro.png" alt="espectro" width="400"/>*

In [ ]:
def exemplo():
    # plota o gráfico x(t) no intervalo [a,a+b] (índices de amostras)
    def iplot(a,b):
        plt.plot(t[a:a+b],x[a:a+b])
        plt.xlabel("tempo (s)")
        plt.title(r"x(t) no intervalo de índices [{},{}]".format(a,a+b))
        plt.show()

    a = IntSlider(min=0, max=100000, step=1000, value=0, continuous_update=False, description=r'$a$')
    b = IntSlider(min=1000, max=10000, step=1000, value=1000, continuous_update=False, description=r'$b$')
    w = interactive(iplot, a=a, b=b)
    display(HBox([w.children[2],VBox([w.children[0],w.children[1]])]))
    w.update()

exemplo()

In [ ]:
# Resposta do exercício 1
def ex1():
    def iplot(m,M):
        M = M if len(x) > m+M else len(x) - m - 1 
        x_cut = x[m:m+M]
        f = fft(x_cut)
        k0 = int(len(x_cut)*2500/R)
        plt.plot(np.arange(k0)*R/len(x_cut),np.abs(f[0:k0]))
        plt.xlabel("Frequência (hz)")
        plt.title(f"Espectro de magnitude [{m}, {m+M})]")
        plt.show()

    m = IntSlider(min=0, max=(R*D)-1, step=1000, value=111000, continuous_update=False, description=r'$m$')
    M = IntSlider(min=100, max=10000, step=100, value=1000, continuous_update=False, description=r'$M$')
    w = interactive(iplot, m=m, M=M)
    display(HBox([w.children[2],VBox([w.children[0],w.children[1]])]))
    w.update()
    
ex1()

#### Resposta do exercício 1 (cont)

1. qual é o perfil típico de cada espectro?
- Cada espectro tem o perfil de um pico em sua frequência.


2. qual é o comportamento aparente em função de $m$, e em particular quais são as frequências mínima e máxima dos picos espectrais observados?
- Quando mudamos o $m$, mudamos a parte do sinal que estamos analisando. A frequência mínima de um pico é algo muito próximo de 0 enquanto a máxima fica em torno de 20000. Como o sinal tem uma frequência crescente (ou seja, vai ficando mais agudo), aumentar o $m$ irá fazendo o pico deslocar para a direita.


3. qual é o comportamento aparente em função de $M$ para um $m$ fixado?
- Para um $m$ fixando, o M aumenta a precisão do programa. Basicamente o programa fica com um pico mais fino, indicando que temos uma certeza maior de qual a frequência que estamos observando.



### Espectrogramas

Uma forma conveniente de organizar os espectros de todos os segmentos $M$-dimensionais de um mesmo sinal $x\in\mathbb{C}^N$ é na forma de uma matriz $\mathcal{X}\in\mathcal{M}_{M\times L}(\mathbb{C})$, onde $L$ é o número de segmentos, e a coluna $m$ contém a DFT de $\tilde{x} = \left(x_{m},x_{m+1},\ldots,x_{m+M-1}\right)\in\mathbb{C}^M$.

Em particular, a matriz $|\mathcal{X}|$ dos valores absolutos com as linhas dispostas ascendentemente de $0$ até $\frac{M}{2}$ (componente de Nyquist) é chamada de *Espectrograma* de $x$. Nessa *imagem*, o tempo é lido na horizontal, a frequência na vertical, e os pixels $(k,m)$ de maior intensidade correspondem às componentes de frequência $k$ ativas no segmento $m$ do sinal.

**Exercício 2:** adapte o código do gráfico interativo para construir um espectrograma do sinal $x$ em função de um parâmetro interativo $M\in\{100,200,300,\ldots,10000\}$, considerando as frequências entre $0$ e $2500$ Hz. Considere que os segmentos são justapostos, começando nas amostras $m=0,M,2M,3M,\ldots$. Comente de forma sucinta na caixa de texto após o código suas observações em relação à visualização oferecida pelo espectrograma para a variação de frequência instantânea do sinal $x$, considerando distintos valores de $M$, e comparando suas observações com aquelas do exercício 1.

**Dica:** *use a opção <tt>aspect='auto'</tt> da função <tt>plt.imshow</tt> para preservar um tamanho fixo do espectrograma para todos os valores de $M$. Um exemplo de espectrograma no formato esperado é <img src="http://www.ime.usp.br/~mqz/dsp/espectrograma.png" alt="espectro" width="400"/>*

In [ ]:
# Resposta do Exercício 2
def ex2():
    def iplot(M):
        # M = M if len(x) > m+M else len(x) - m - 1
        L = len(x)//M
        sg = np.zeros((M//2, L))
        k0 = int(M*2500/R)
        for i in range(0, L):
            m = i*M
            x_cut = x[m:m+M]
            f = fft(x_cut)
            sg[:, i] = np.abs(f[0:M//2])
        plt.imshow(sg[0:k0, :], aspect='auto', origin='lower', extent=(0.0, 10.0, 0, 2500))
        plt.show()

    M = IntSlider(min=100, max=10000, step=100, value=1000, continuous_update=False, description=r'$M$')
    w = interactive(iplot, M=M)
    display(HBox([w.children[1],VBox([w.children[0]])]))
    w.update()
    
    
ex2()

#### Resposta do exercício 2 (cont)

**Comentários e observações:**

O espectrograma nos permite visualizar o gráfico que fizemos no anterior porém para todos os valores de $m$ simultaneamente para um $M$ fixo. Ou seja, fica mais fácil de verificar as alterações da frequência do sinal de acordo com o tempo. Também podemos perceber, que quando aumentamos o $M$, o gráfico fica mais nítido, indicando (igual no exercício anterior), que estamos mais certos da frequência sendo verificada.

### Frequência instantânea

Afinal como se estima a frequência instantânea de um sinal senoidal de frequência variável?

E porque a expressão 

$$x(t) = \sin(2\pi f(t)t),\quad t\in[0,D)$$

com $f(t)$ indo de $0$ até $1000$ produz frequências instantâneas de até $2000$ Hz?

Uma maneira de conceituar a frequência instantânea de um sinal senoidal é considerá-lo como projeção de um movimento circular uniforme: por exemplo, a função complexa $g(t)=e^{i2\pi ft}$ possui como projeção horizontal (eixo real) a função $g_1(t)=\Re[g(t)]=\cos(2\pi ft)$ e como projeção vertical a função $g_2(t)=\Im[g(t)]=\sin(2\pi ft)$.

Nas duas projeções, a *velocidade de giro* é de $\omega=2\pi f$ radianos/segundo, ou $f=\frac{\omega}{2\pi}$ Hz, que correspondem ao ângulo ou fase (em radianos ou ciclos) percorrido num intervalo de 1 segundo. 

Em outras palavras, a frequência (angular) instantânea do sinal $x(t)$ corresponde à variação de fase
$$\tilde{\omega} = \frac{\varphi_2-\varphi_1}{t_2-t_1} = \frac{2\pi f(t_2)t_2-2\pi f(t_1)t_1}{t_2-t_1}.$$
para dois instantes sucessivos $t_1$ e $t_2$ bem próximos (no limite, estaríamos falando da *derivada* da fase instantânea).

Num sinal digital amostrado a $R$ Hz, essa estimativa de frequência instantânea pode ser feita a cada amostra, considerando-se a expressão acima para dois instantes amostrais sucessivos $t_1$ e $t_2 = t_1+\frac{1}{R}$.

A estimativa $\tilde{\omega}$ estará em radianos/segundo, mas pode ser convertida facilmente para Hz usando a expressão $\tilde{f} = \frac{\tilde{\omega}}{2\pi}$.

**Exercício 3:** Escreva um código que calcule a frequência instantânea em Hz $\tilde{f}(t)$ do sinal $x$ a partir da definição acima, e plote em um mesmo gráfico as funções $f(t)$ e $\tilde{f}(t)$ no intervalo $t\in[0,10)$. Comente após o código: seu gráfico corrobora as observações dos exercícios 1 e 2?

In [ ]:
# Resposta do exercício 3
omega_til = lambda t : 2*np.pi*R*(100*(t+1/R)*(t+1/R) - 100*t*t)
t = np.arange(0, 10, 1/R)

f = 100*t
f_til = omega_til(t)/(2*np.pi)

plt.plot(t, f, label='Frequência original')
plt.plot(t, f_til, label='Frequência instantânea')
plt.legend()
plt.show()

**Resposta do exercício 3 (cont)**

**Comente:** Seu gráfico corrobora as observações dos exercícios 1 e 2?

Sim, corrobora. Nos exercícios anteriores, quanto menor o $M$, mais próximo estávamos de pegar a frequência instantânea. Por exemplo: no exercício 1, se colocarmos o $M$ no mínimo e o $m$ no máximo, podemos lembrar que conseguíamos um pico no 2000. O gráfico acima mostra que a frequência instantânea no tempo final é 2000 também. 